This can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/pipeline-orchestrators/orchestration_sdk_comprehensive_tutorial).

## Pipeline Orchestration Comprehensive Tutorial

This tutorial provides a complete set of methods and examples regarding Wallaroo Connections and Wallaroo ML Workload Orchestration.

Wallaroo provides data connections, orchestrations, and tasks to provide organizations with a method of creating and managing automated tasks that can either be run on demand, on a regular schedule, or as a service so they respond to requests.

| Object | Description |
|---|---|
| Orchestration | A set of instructions written as a python script with a requirements library.  Orchestrations are uploaded to the Wallaroo instance |
| Task | An implementation of an orchestration.  Tasks are run either once when requested, on a repeating schedule, or as a service. |
| Connection | Definitions set by MLOps engineers that are used by other Wallaroo users for connection information to a data source.  Usually paired with orchestrations. |

A typical flow in the orchestration, task and connection life cycle is:

1. (Optional) A connection is defined with information such as username, connection URL, tokens, etc.
1. One or more connections are applied to a workspace for users to implement in their code or orchestrations.
1. An orchestration is created to perform some set instructions.  For example:
    1. Deploy a pipeline, request data from an external service, store the results in an external database, then undeploy the pipeline.
    1. Download a ML Model then replace a current pipeline step with the new version.
    1. Collect log files from a deployed pipeline once every hour and submit it to a Kafka or other service.
1. A task is created that specifies the orchestration to perform and the schedule:
    1. Run once.
    1. Run on a schedule (based on `cron` like settings).
    1. Run as a service to be run whenever requested.
1. Once the use for a task is complete, it is killed and its schedule or service removed.

## Tutorial Goals

The tutorial will demonstrate the following:

1. Create a simple connection to retrieve an Apache Arrow table file from a GitHub registry.
1. Create an orchestration that retrieves the Apache Arrow table file from the location defined by the connection, deploy a pipeline, perform an inference, then undeploys the pipeline.
1. Implement the orchestration as a task that runs every minute.
1. Display the logs from the pipeline after 5 minutes to verify the task is running.

## Tutorial Required Libraries

The following libraries are required for this tutorial, and included by default in a Wallaroo instance's JupyterHub service.

* **IMPORTANT NOTE**:  These libraries are already installed in the Wallaroo JupyterHub service.  Do not uninstall and reinstall the Wallaroo SDK with the command below.

* [wallaroo](https://pypi.org/project/wallaroo/):  The Wallaroo SDK.
* [pandas](https://pypi.org/project/pandas/): The pandas data analysis library.
* [pyarrow](https://pypi.org/project/pyarrow/): The Apache Arrow Python library.

The specific versions used are set in the file `./resources/requirements.txt`.  Supported libraries are automatically installed with the `pypi` or `conda` commands.  For example, from the root of this tutorials folder:

```python
pip install -r ./resources/requirements.txt
```

## Initialization

The first step is to connect to a Wallaroo instance.  We'll load the libraries and set our client connection settings

### Workspace, Model and Pipeline Setup

For this tutorial, we'll create a workspace, upload our sample model and deploy a pipeline.  We'll perform some quick sample inferences to verify that everything it working.

In [65]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# to display dataframe tables
from IPython.display import display
# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pyarrow as pa

import requests

# Used to create unique workspace and pipeline names
import string
import random

# make a random 4 character prefix
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))
display(suffix)

'mrhh'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [ ]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

In [67]:
# Setting variables for later steps

workspace_name = f'orchestrationworkspace{suffix}'
pipeline_name = f'orchestrationpipeline{suffix}'
model_name = f'orchestrationmodel{suffix}'
model_file_name = './models/rf_model.onnx'
connection_name = f'houseprice_arrow_table{suffix}'

### Helper Methods

The following helper methods are used to either create or get workspaces and pipelines.

In [68]:
# helper methods to retrieve workspaces and pipelines

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

### Create the Workspace and Pipeline

We'll now create our workspace and pipeline for the tutorial.  If this tutorial has been run previously, then this will retrieve the existing ones with the assumption they're for us with this tutorial.

We'll set the retrieved workspace as the current workspace in the SDK, so all commands will default to that workspace.

In [69]:
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload the Model and Deploy Pipeline

We'll upload our model into our sample workspace, then add it as a pipeline step before deploying the pipeline to it's ready to accept inference requests.

In [70]:
# Upload the model

housing_model_control = wl.upload_model(model_name, model_file_name).configure()

# Add the model as a pipeline step

pipeline.add_model_step(housing_model_control)

name,orchestrationpipelinemrhh
created,2023-05-19 20:50:05.233902+00:00
last_updated,2023-05-19 20:50:05.233902+00:00
deployed,(none)
tags,
versions,1382876a-eeb4-4f50-817b-5bf0604f21d4
steps,


In [71]:
#deploy the pipeline
pipeline.deploy()

name,orchestrationpipelinemrhh
created,2023-05-19 20:50:05.233902+00:00
last_updated,2023-05-19 20:50:08.177531+00:00
deployed,True
tags,
versions,"9a5c2d59-48da-40ff-b7ae-9f3a529b9aaf, 1382876a-eeb4-4f50-817b-5bf0604f21d4"
steps,orchestrationmodelmrhh


### Sample Inferences

We'll perform some quick sample inferences using an Apache Arrow table as the input.  Once that's finished, we'll undeploy the pipeline and return the resources back to the Wallaroo instance.

In [72]:
# sample inferences

batch_inferences = pipeline.infer_from_file('./data/xtest-1k.arrow')

large_inference_result =  batch_inferences.to_pandas()
display(large_inference_result.head(20))

,time,in.tensor,out.variable,check_failures
0,2023-05-19 20:50:24.194,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-05-19 20:50:24.194,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-05-19 20:50:24.194,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-05-19 20:50:24.194,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-05-19 20:50:24.194,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
5,2023-05-19 20:50:24.194,"[3.0, 2.0, 2140.0, 4923.0, 1.0, 0.0, 0.0, 4.0, 8.0, 1070.0, 1070.0, 47.6902, -122.339, 1470.0, 4923.0, 86.0, 0.0, 0.0]",[668288.0],0
6,2023-05-19 20:50:24.194,"[4.0, 3.5, 3590.0, 5334.0, 2.0, 0.0, 2.0, 3.0, 9.0, 3140.0, 450.0, 47.6763, -122.267, 2100.0, 6250.0, 9.0, 0.0, 0.0]",[1004846.5],0
7,2023-05-19 20:50:24.194,"[3.0, 2.0, 1280.0, 960.0, 2.0, 0.0, 0.0, 3.0, 9.0, 1040.0, 240.0, 47.602, -122.311, 1280.0, 1173.0, 0.0, 0.0, 0.0]",[684577.2],0
8,2023-05-19 20:50:24.194,"[4.0, 2.5, 2820.0, 15000.0, 2.0, 0.0, 0.0, 4.0, 9.0, 2820.0, 0.0, 47.7255, -122.101, 2440.0, 15000.0, 29.0, 0.0, 0.0]",[727898.1],0
9,2023-05-19 20:50:24.194,"[3.0, 2.25, 1790.0, 11393.0, 1.0, 0.0, 0.0, 3.0, 8.0, 1790.0, 0.0, 47.6297, -122.099, 2290.0, 11894.0, 36.0, 0.0, 0.0]",[559631.1],0


## Create Wallaroo Connection




Connections are created at the Wallaroo instance level, typically by a MLOps or DevOps engineer, then applied to a workspace.

For this section:

1. We will create a sample connection that just has a URL to the same Arrow table file we used in the previous step.
1. We'll apply the data connection to the workspace above.
1. For a quick demonstration, we'll use the connection to retrieve the Arrow table file and use it for a quick sample inference.

### Create Connection

Connections are created with the Wallaroo client command [`create_connection`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#create-data-connection) with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the connection. This must be unique - if submitting the name of an existing connection it will return an error. |
| **type** | string (Required) | The user defined type of connection. |
| **details** | Dict (Requires) | User defined configuration details for the data connection.  These can be `{'username':'dataperson', 'password':'datapassword', 'port': 3339}`, or `{'token':'abcde123==', 'host':'example.com', 'port:1234'}`, or other user defined combinations.  |

We'll create the connection named `houseprice_arrow_table`, set it to the type `HTTPFILE`, and provide the details as `'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'` - the location for our sample Arrow table inference input.

In [73]:
wl.create_connection(connection_name, 
                  "HTTPFILE", 
                  {'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'}
                  )

Field,Value
Name,houseprice_arrow_tablemrhh
Connection Type,HTTPFILE
Details,*****
Created At,2023-05-19T20:50:24.878300+00:00
Linked Workspaces,[]


### List Data Connections

The Wallaroo Client [`list_connections()`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#list-data-connections) method lists all connections for the Wallaroo instance.


In [74]:
wl.list_connections()

name,connection type,details,created at,linked workspaces
houseprice_arrow_tablemrhh,HTTPFILE,*****,2023-05-19T20:50:24.878300+00:00,[]


### Add Connection to Workspace

The method Workspace [`add_connection(connection_name)`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#add-data-connection-to-workspace) adds a Data Connection to a workspace, and takes the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the Data Connection |

We'll add this connection to our sample workspace.

In [75]:
workspace.add_connection(connection_name)

### List Connections in a Workspace

The method Workspace `list_connections()` displays a list of connections attached to the workspace.  By default the `details` field is obfuscated.  We'll list the connections in our sample workspace, then use that to retrieve our specific connection.

In [76]:
workspace.list_connections()

name,connection type,details,created at,linked workspaces
houseprice_arrow_tablemrhh,HTTPFILE,*****,2023-05-19T20:50:24.878300+00:00,['orchestrationworkspacemrhh']


### Get Connection

Connections are retrieved by the Wallaroo Client `get_connection(name)` method.

In [77]:
connection = wl.get_connection(connection_name)

### Connection Details

The Connection method `details()` retrieves a the connection `details()` as a `dict`.

In [78]:
display(connection.details())

{'host': 'https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20230314_2023.2_updates/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'}

### Using a Connection Example

For this example, the connection will be used to retrieve the Apache Arrow file referenced in the connection, and use that to turn it into an Apache Arrow table, then use that for a sample inference.

In [79]:
# Deploy the pipeline 
pipeline.deploy()

# Retrieve the file
# set accept as apache arrow table
headers = {
    'Accept': 'application/vnd.apache.arrow.file'
}

response = requests.get(
                    connection.details()['host'], 
                    headers=headers
                )

# Arrow table is retrieved 
with pa.ipc.open_file(response.content) as reader:
    arrow_table = reader.read_all()

results = pipeline.infer(arrow_table)

result_table = results.to_pandas()
display(result_table.head(20))

,time,in.tensor,out.variable,check_failures
0,2023-05-19 20:50:30.348,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-05-19 20:50:30.348,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-05-19 20:50:30.348,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-05-19 20:50:30.348,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-05-19 20:50:30.348,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
5,2023-05-19 20:50:30.348,"[3.0, 2.0, 2140.0, 4923.0, 1.0, 0.0, 0.0, 4.0, 8.0, 1070.0, 1070.0, 47.6902, -122.339, 1470.0, 4923.0, 86.0, 0.0, 0.0]",[668288.0],0
6,2023-05-19 20:50:30.348,"[4.0, 3.5, 3590.0, 5334.0, 2.0, 0.0, 2.0, 3.0, 9.0, 3140.0, 450.0, 47.6763, -122.267, 2100.0, 6250.0, 9.0, 0.0, 0.0]",[1004846.5],0
7,2023-05-19 20:50:30.348,"[3.0, 2.0, 1280.0, 960.0, 2.0, 0.0, 0.0, 3.0, 9.0, 1040.0, 240.0, 47.602, -122.311, 1280.0, 1173.0, 0.0, 0.0, 0.0]",[684577.2],0
8,2023-05-19 20:50:30.348,"[4.0, 2.5, 2820.0, 15000.0, 2.0, 0.0, 0.0, 4.0, 9.0, 2820.0, 0.0, 47.7255, -122.101, 2440.0, 15000.0, 29.0, 0.0, 0.0]",[727898.1],0
9,2023-05-19 20:50:30.348,"[3.0, 2.25, 1790.0, 11393.0, 1.0, 0.0, 0.0, 3.0, 8.0, 1790.0, 0.0, 47.6297, -122.099, 2290.0, 11894.0, 36.0, 0.0, 0.0]",[559631.1],0


### Remove Connection from Workspace

The Workspace method `remove_connection(connection_name)` removes the connection from the workspace, but does not delete the connection from the Wallaroo instance.  This method takes the following parameters.

| Parameter | Type | Description |
|---|---|---|
| **name** | String (Required) | The name of the connection to be removed 

The previous connection will be removed from the workspace, then the workspace connections displayed to verify it has been removed.

In [80]:
workspace.remove_connection(connection_name)

display(workspace.list_connections())

[]

### Delete Connection

The Connection method `delete_connection()` removes the connection from the Wallaroo instance, and all attachments in workspaces they were connected to.

In [81]:
connection.delete_connection()

wl.list_connections()

[]

## Orchestration Tutorial

The next series of examples will build on what we just did.  So far we have:

* Deployed a pipeline, performed sample inferences with a local Apache Arrow file, displayed the results, then undeployed the pipeline.
* Deployed a pipeline, use a Wallaroo connection details to retrieve a remote Apache Arrow file, performed inferences and displayed the results, then undeployed the pipeline.

For the orchestration tutorial, we'll do the same thing only package it into a separate python script and upload it to the Wallaroo instance, then create a task from that orchestration and perform our sample inferences again.

### Orchestration Requirements

Orchestrations are uploaded to the Wallaroo instance as a ZIP file with the following requirements:

* The ZIP file should not contain any directories - only files at the top level.

| Parameter | Type | Description |
|---|---|---|
| **User Code** | (**Required**) Python script as `.py` files | Python scripts for the orchestration to run.  If the file `main.py` exists, that will be the entrypoint.  Otherwise, if only one .py exists, then that will be the entrypoint. |
| Python Library Requirements | (**Required**) `requirements.txt` file in the [requirements file format](https://pip.pypa.io/en/stable/reference/requirements-file-format/).  This is in the root of the zip file, and there can only be one `requirements.txt` file for the orchestration. |
| Other artifacts | &nbsp; | Other artifacts such as files, data, or code to support the orchestration.

#### Zip Instructions

In a terminal with the `zip` command, assemble artifacts as above and then create the archive.  The `zip` command is included by default with the Wallaroo JupyterHub service.

`zip` commands take the following format, with `{zipfilename}.zip` as the zip file to save the artifacts to, and each file thereafter as the files to add to the archive.

```bash
zip {zipfilename}.zip file1, file2, file3....
```

For example, the following command will add the files `main.py` and `requirements.txt` into the file `hello.zip`.

```shell
$ zip hello.zip main.py requirements.txt 
  adding: main.py (deflated 47%)
  adding: requirements.txt (deflated 52%)
```

### Orchestration Recommendations

The following recommendations will make using Wallaroo orchestrations 

* The version of Python used should match the same version as in the Wallaroo JupyterHub service.
* The same version of the [Wallaroo SDK](https://pypi.org/project/wallaroo/) should match the server.  For a 2023.2 Wallaroo instance, use the Wallaroo SDK version 2023.2.
* Specify the version of `pip` dependencies.
* The `wallaroo.Client` constructor `auth_type` argument is ignored.  Using `wallaroo.Client()` is sufficient.
* The following methods will assist with orchestrations:
  * `wallaroo.in_task()` :  Returns `True` if the code is running within an Orchestrator task.
  * `wallaroo.task_args()`:  Returns a `Dict` of invocation-specific arguments passed to the `run_` calls.
* Use `print` commands so outputs are saved to the task's log files.

### Example requirements.txt file

```python
dbt-bigquery==1.4.3
dbt-core==1.4.5
dbt-extractor==0.4.1
dbt-postgres==1.4.5
google-api-core==2.8.2
google-auth==2.11.0
google-auth-oauthlib==0.4.6
google-cloud-bigquery==3.3.2
google-cloud-bigquery-storage==2.15.0
google-cloud-core==2.3.2
google-cloud-storage==2.5.0
google-crc32c==1.5.0
google-pasta==0.2.0
google-resumable-media==2.3.3
googleapis-common-protos==1.56.4
```

### Sample Orchestrator

The following orchestrator artifacts are in the directory `./remote_inference` and includes the file `main.py` with the following code:

```python
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas as pd
import pyarrow as pa
import requests

wl = wallaroo.Client()

# Setting variables for later steps

# get the arguments
arguments = wl.task_args()

if "workspace_name" in arguments:
    workspace_name = arguments['workspace_name']
else:
    workspace_name="orchestrationworkspace"

if "pipeline_name" in arguments:
    pipeline_name = arguments['pipeline_name']
else:
    pipeline_name="orchestrationpipeline"

if "connection_name" in arguments:
    connection_name = arguments['connection_name']
else:
    connection_name = "houseprice_arrow_table"


# helper methods to retrieve workspaces and pipelines

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

print(f"Getting the workspace {workspace_name}")
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)


print(f"Getting the pipeline {pipeline_name}")
pipeline = get_pipeline(pipeline_name)
pipeline.deploy()
# Get the connection - assuming it will be the only one

inference_source_connection = wl.get_connection(name=connection_name)

print(f"Getting arrow table file")
# Retrieve the file
# set accept as apache arrow table
headers = {
    'Accept': 'application/vnd.apache.arrow.file'
}

response = requests.get(
                    inference_source_connection.details()['host'], 
                    headers=headers
                )

# Arrow table is retrieved 
with pa.ipc.open_file(response.content) as reader:
    arrow_table = reader.read_all()

print("Inference time.  Displaying results after.")
# Perform the inference
result = pipeline.infer(arrow_table)
print(result)

pipeline.undeploy()
```

This is saved to the file `./remote_inference/remote_inference.zip`.

### Preparing the Wallaroo Instance

To prepare the Wallaroo instance, we'll once again create the Wallaroo connection `houseprice_arrow_table` and apply it to the workspace.

In [82]:
wl.create_connection(connection_name, 
                  "HTTPFILE", 
                  {'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'}
                  )

workspace.add_connection(connection_name)

### Upload the Orchestration

Orchestrations are uploaded with the Wallaroo client `upload_orchestration(path)` method with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **path** | string (Required) | The path to the ZIP file to be uploaded. |

Once uploaded, the deployment will be prepared and any requirements will be downloaded and installed.


For this example, the orchestration `./remote_inference/remote_inference.zip` will be uploaded and saved to the variable `orchestration`.

In [83]:
orchestration = wl.upload_orchestration(name="comprehensive sample", path="./remote_inference/remote_inference.zip")

### Orchestration Status

The Orchestration method `status()` displays the current status of the uploaded orchestration.

| Status | Description |
|---|---|
|---|---|
| `pending_packaging` | The orchestration is uploaded, but packaging hasn't started yet. |
| `packaging` | The orchestration is being packaged for use with the Wallaroo instance. |
| `ready` | The orchestration is ready for use. |

For this example, the status of the orchestration will be displayed then looped until it has reached status `ready`.

In [84]:
import time

while orchestration.status() != 'ready':
    print(orchestration.status())
    time.sleep(5)

pending_packaging
pending_packaging
packaging
packaging
packaging
packaging
packaging
packaging
packaging
packaging


### List Orchestrations

Orchestrations are listed with the Wallaroo Client `list_orchestrations()` which returns a list of available orchestrations.

In [85]:
wl.list_orchestrations()

id,name,status,filename,sha,created at,updated at
9534a646-254e-4d86-a577-5c40e5defeb3,comprehensive sample,ready,remote_inference.zip,b88e93...2396fb,2023-19-May 20:50:33,2023-19-May 20:51:24


## Task Management Tutorial

Once an Orchestration has the status `ready`, it can be run as a task.  Tasks have three run options.

| Type       | SDK Call |  How triggered                                                               | Purpose                                                       |
|------------|----------|:------------------------------------------------------------------------------|:---------------------------------------------------------------|
| Once       | `orchestration.run_once(name, json_args, timeout)` | Task runs once and exits.| Single batch, experimentation                                 |
| Scheduled  | `orchestration.run_scheduled()` | User provides schedule. Task runs exits whenever schedule dictates. | Recurrent batch ETL.|

### Task Run Once

Tasks are generated and run once with the Orchestration `run_once(name, json_args, timeout)` method.  Any arguments for the orchestration are passed in as a `Dict`.  If there are no arguments, then an empty set `{}` is passed.

For our example, we will pass the workspace, pipeline, and connection into our task.

In [86]:
# Example: run once

import datetime
task_start = datetime.datetime.now()

task = orchestration.run_once(name="house price run once", json_args={"workspace_name": workspace_name, 
                                                                           "pipeline_name":pipeline_name,
                                                                           "connection_name": connection_name
                                                                           }
                            )
task

Field,Value
ID,7bce29ee-70c7-4c59-b7ec-0c9941073883
Name,house price run once
Status,pending
Type,Temporary Run
Created At,2023-19-May 20:51:31
Updated At,2023-19-May 20:51:31


### Task Status

The list of tasks in the Wallaroo instance is retrieves through the Wallaroo Client `list_tasks()` method.  This returns an array list of the following.

| Parameter | Type | Description |
| --- | --- | ---|
| **id** | string | The UUID identifier for the task. |
| **status** | enum | The status of the task.  Values are: <br><ul><li>`pending`: The task has not been started.</li><li>`started`: The task has been scheduled to execute.</li><li>`pending_kill`: The task kill command has been issued and the task is scheduled to be stopped.</li></ul> |
| **type** | string | The type of the task.  Values are: <br><ul><li>`Temporary Run`: The task runs once then stop.</li><li>`Scheduled Run`: The task repeats on a `cron` like schedule.</li><li>`Service Run`: The task runs as a service and executes when its service port is activated. |
| **created at** | DateTime | The date and time the task was started. |
| **updated at** | DateTime | The date and time the task was updated. |

For this example, the status of the previously created task will be generated, then looped until it has reached status `started`.

In [87]:
while task.status() != "started":
    display(task.status())
    time.sleep(5)

'pending'

'pending'

### Task Results

We can view the inferences from our logs and verify that new entries were added from our task.  In our case, we'll assume the task once started takes about 1 minute to run (deploy the pipeline, run the inference, undeploy the pipeline).  We'll add in a wait of 1 minute, then display the logs during the time period the task was running.

In [88]:
time.sleep(60)

task_end = datetime.datetime.now()
display(task_end)

pipeline.logs(start_datetime = task_start, end_datetime = task_end)

datetime.datetime(2023, 5, 19, 14, 52, 43, 243562)

,time,in.tensor,out.variable,check_failures
0,2023-05-19 20:51:43.045,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-05-19 20:51:43.045,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-05-19 20:51:43.045,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-05-19 20:51:43.045,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-05-19 20:51:43.045,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
...,...,...,...,...
501,2023-05-19 20:51:43.045,"[3.0, 2.5, 1570.0, 1433.0, 3.0, 0.0, 0.0, 3.0, 8.0, 1570.0, 0.0, 47.6858, -122.336, 1570.0, 2652.0, 4.0, 0.0, 0.0]",[557391.25],0
502,2023-05-19 20:51:43.045,"[3.0, 2.5, 2390.0, 15669.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2390.0, 0.0, 47.7446, -122.193, 2640.0, 12500.0, 24.0, 0.0, 0.0]",[741973.6],0
503,2023-05-19 20:51:43.045,"[3.0, 0.75, 920.0, 20412.0, 1.0, 1.0, 2.0, 5.0, 6.0, 920.0, 0.0, 47.4781, -122.49, 1162.0, 54705.0, 64.0, 0.0, 0.0]",[338418.8],0
504,2023-05-19 20:51:43.045,"[4.0, 2.5, 2800.0, 246114.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2800.0, 0.0, 47.6586, -121.962, 2750.0, 60351.0, 15.0, 0.0, 0.0]",[765468.75],0


### Scheduled Tasks

Scheduled tasks are run with the Orchestration `run_scheduled` method.  We'll set it up to run every 5 minutes, then check the results.

It is recommended that orchestrations that have pipeline deploy or undeploy commands be spaced out no less than 5 minutes to prevent colliding with other tasks that use the same pipeline.

In [89]:
task_start = datetime.datetime.now()
schedule = "*/5 * * * *"
task_scheduled = orchestration.run_scheduled(name="schedule example", 
                                             timeout=600, 
                                             schedule=schedule, 
                                             json_args={"workspace_name": workspace_name, 
                                                        "pipeline_name": pipeline_name,
                                                        "connection_name": connection_name
                                            })
while task_scheduled.status() != "started":
    display(task_scheduled.status())
    time.sleep(5)
task_scheduled

Field,Value
ID,487fe7c0-c818-4156-b238-71da4aa9da87
Name,schedule example
Status,started
Type,Scheduled Run
Created At,2023-19-May 20:52:44
Updated At,2023-19-May 20:52:45


In [90]:
time.sleep(420)

task_end = datetime.datetime.now()
display(task_end)

pipeline.logs(start_datetime = task_start, end_datetime = task_end)

datetime.datetime(2023, 5, 19, 14, 59, 45, 925180)

,time,in.tensor,out.variable,check_failures
0,2023-05-19 20:55:19.922,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-05-19 20:55:19.922,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-05-19 20:55:19.922,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-05-19 20:55:19.922,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-05-19 20:55:19.922,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
...,...,...,...,...
501,2023-05-19 20:55:19.922,"[3.0, 2.5, 1570.0, 1433.0, 3.0, 0.0, 0.0, 3.0, 8.0, 1570.0, 0.0, 47.6858, -122.336, 1570.0, 2652.0, 4.0, 0.0, 0.0]",[557391.25],0
502,2023-05-19 20:55:19.922,"[3.0, 2.5, 2390.0, 15669.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2390.0, 0.0, 47.7446, -122.193, 2640.0, 12500.0, 24.0, 0.0, 0.0]",[741973.6],0
503,2023-05-19 20:55:19.922,"[3.0, 0.75, 920.0, 20412.0, 1.0, 1.0, 2.0, 5.0, 6.0, 920.0, 0.0, 47.4781, -122.49, 1162.0, 54705.0, 64.0, 0.0, 0.0]",[338418.8],0
504,2023-05-19 20:55:19.922,"[4.0, 2.5, 2800.0, 246114.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2800.0, 0.0, 47.6586, -121.962, 2750.0, 60351.0, 15.0, 0.0, 0.0]",[765468.75],0


### Kill a Task

Killing a task removes the schedule or removes it from a service.  Tasks are killed with the Task `kill()` method, and returns a message with the status of the kill procedure.

If necessary, all tasks can be killed through the following script.

* **IMPORTANT NOTE**:  This command will kill all running tasks - scheduled or otherwise.  Only use this if required.

```python
# Kill all tasks
for t in wl.list_tasks(): t.kill()
```

In [91]:
task_scheduled.kill()

<ArbexStatus.PENDING_KILL: 'pending_kill'>

## Cleaning Up

With the tutorial complete we will undeploy the pipeline and ensure the resources are returned back to the Wallaroo instance.

In [92]:
pipeline.undeploy()

name,orchestrationpipelinemrhh
created,2023-05-19 20:50:05.233902+00:00
last_updated,2023-05-19 20:50:27.486572+00:00
deployed,False
tags,
versions,"b72281d7-e501-4d45-8a79-4c724b639c2f, 9a5c2d59-48da-40ff-b7ae-9f3a529b9aaf, 1382876a-eeb4-4f50-817b-5bf0604f21d4"
steps,orchestrationmodelmrhh
